In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce


In [2]:
# Alpaca API credentials
api_key = "PK2DSD4BN8QNOYWSC5I1"
secret_key = "eGSHvlwHLcuf24k0cTKXDPYFMjs3n1per4SsAys8"

In [3]:
client = StockHistoricalDataClient(api_key, secret_key)
trading_client = TradingClient(api_key, secret_key, paper=True)

In [4]:
def compute_macd(data, fast=12, slow=26, signal=9):
    data['ema_fast'] = data['close'].ewm(span=fast, adjust=False).mean()
    data['ema_slow'] = data['close'].ewm(span=slow, adjust=False).mean()
    data['macd'] = data['ema_fast'] - data['ema_slow']
    data['signal'] = data['macd'].ewm(span=signal, adjust=False).mean()
    data['histogram'] = data['macd'] - data['signal']
    return data

In [5]:
# Fetch historical stock data for MAG6
symbols = ["AAPL", "GOOGL", "AMZN", "META", "MSFT", "NVDA"]
start_date = dt.datetime.now() - dt.timedelta(days=365)
end_date = dt.datetime.now()

request_params = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame.Day,
    start=start_date,
    end=end_date
)

bars = client.get_stock_bars(request_params).df
stock_dic = {}
scaler = MinMaxScaler(feature_range=(0, 1))

for symbol in symbols:
    df = bars[bars.index.get_level_values(0) == symbol].copy()
    df.reset_index(inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df = compute_macd(df)
    df[['close', 'macd', 'signal']] = scaler.fit_transform(df[['close', 'macd', 'signal']])
    stock_dic[symbol] = df

In [6]:
# Predict future prices and execute trades
predictions = {}
capital = 10000  # Example starting capital
position_size = 0.1  # Allocate 10% of capital per trade

for symbol in symbols:
    test_data = stock_dic[symbol][['close', 'macd', 'signal']].values
    X_test, y_test = create_sequences(test_data)
    pred = model.predict(X_test)
    predictions[symbol] = pred
    
    last_actual_price = stock_dic[symbol]['close'].iloc[-1]
    last_predicted_price = pred[-1][0]
    
    qty = int((capital * position_size) / last_actual_price)  # Determine quantity
    
    if last_predicted_price > last_actual_price and qty > 0:  # Buy condition
        order = MarketOrderRequest(
            symbol=symbol,
            qty=qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC
        )
        trading_client.submit_order(order)
        print(f"BUY Order placed for {symbol}: {qty} shares")
    elif last_predicted_price < last_actual_price and qty > 0:  # Sell condition
        order = MarketOrderRequest(
            symbol=symbol,
            qty=qty,
            side=OrderSide.SELL,
            time_in_force=TimeInForce.GTC
        )
        trading_client.submit_order(order)
        print(f"SELL Order placed for {symbol}: {qty} shares")

# Print trade results
print("Trading bot executed based on LSTM predictions and MACD analysis.")


NameError: name 'create_sequences' is not defined